<a href="https://colab.research.google.com/github/dashadem/HPC-project/blob/main/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuda realization

In [ ]:
%%writefile HPCProject.cu

#include <stdio.h>
#include <iostream>
#include <array>

const int Nh = 1000;
const int Nt = 100000;
const float h = (float) 1 / (Nh - 1);
const double t = 0.00000001;
const float tau = (float) t/Nt;
const float B = 0.;

// GPU code

__device__ int getGid() {

    int tid = threadIdx.x + blockDim.x * threadIdx.y +
              blockDim.x * blockDim.y * threadIdx.z;

    int bid = blockIdx.x + gridDim.x * blockIdx.y +
              gridDim.x * gridDim.y * blockIdx.z;

    int gid = bid * blockDim.x * blockDim.y * blockDim.z + tid;

    return gid;
}

__global__ void initVars(float* gamma, float* v, float*omega){
    unsigned int gid = getGid();
    if (gid == 0){
      gamma[0] = 0.1;
      v[0] = pow(omega[Nh-1],2)*omega[Nh-2]/(gamma[0])/2/h;
    }
}

__global__ void updateVars(float* gamma, float* v, float*omega){
    unsigned int gid = getGid();
    if (gid == 0){
      v[0] = -pow(omega[Nh-1],2)/gamma[0]*((omega[Nh] - omega[Nh-2])/2/h - gamma[0]*B);
      gamma[0] = gamma[0] + tau*v[0];
    }
}

__global__ void printMatrix(float* array, int size) {
    unsigned int gid = getGid();
    if (gid < size) {
      printf("Gid = %d, value = %f \n", gid, array[gid]);
    }
}

__global__ void initKsi(float* ksi, int size) {
    int gid = getGid();
    if (gid < size) {
        ksi[gid] = 1 / (float)(size - 1) * gid;
    }
}

__global__ void initOmega(float* omega,  int size, float* ksi) {
    int gid = getGid();
    if (gid < size) {
        omega[gid] = sqrt(1 - ksi[gid]);
    }
}

__global__ void printArray(float* array, int size) {
    int gid = getGid();
    if (gid < size) {
        printf("I am %d, I have a value %.10f \n", gid,array[gid]);
    }
}

__global__ void iterate(float* omega, float* ksi, int size, float*gamma, float* v) {
    int gid = getGid();
    if (gid < size) {
      for (int i = 0; i<Nt; i++){
        if ((gid >0) && (gid < size-1)){
          float value = omega[gid] + tau*((1/gamma[0]*v[0]*ksi[gid])*(omega[gid + 1] - omega[gid])/h -
                                        1/gamma[0]/h * ( -pow(omega[gid+1] + omega[gid], 3)/8/gamma[0] * ((omega[gid+1] - omega[gid])/h - gamma[0] * B) +
                                                    pow(omega[gid-1] + omega[gid],3)/8/gamma[0] * ((omega[gid] - omega[gid-1])/h - gamma[0] * B)));
          __syncthreads();
          omega[gid] = value;
        }
        if (gid == 0){
          v[0] = -pow(omega[Nh-1],2)/gamma[0]*((omega[Nh] - omega[Nh-2])/2/h - gamma[0]*B);
          gamma[0] = gamma[0] + tau*v[0];
        }
        //__syncthreads();
      }
    }
}

// CPU code
int main() {

     FILE *file;

    // Open a file in writing mode
    file = fopen("cudaSolution.txt", "w");


    std::array<float, Nh> results;
    for(auto n = 0; n < Nh; ++n) {
        results[n] = 0.;
    }

    float* ksi;
    cudaMalloc(&ksi, Nh*sizeof(float));
    initKsi<<<32, 32>>>(ksi, Nh);

    float* omega;
    cudaMalloc(&omega, Nh*sizeof(float));
    initOmega<<<32, 32>>>(omega, Nh, ksi);

    float* gamma;
    cudaMalloc(&gamma, sizeof(float));

    float* v;
    cudaMalloc(&v, sizeof(float));
    initVars<<<32, 32>>>(gamma, v, omega);

    iterate<<<32, 32>>>(omega, ksi, Nh, gamma, v);

    cudaMemcpy(results.data(), omega, Nh * sizeof(float), cudaMemcpyDeviceToHost);

    for (int i = 0; i<Nh; i++){
        fprintf(file, "%f \n", results[i]);
    }


    // Close the file
    fclose(file);

    cudaDeviceSynchronize();
    std::cout << "Done from CPU\n";
    return 0;
}

Overwriting HPCProject.cu


In [ ]:
!nvcc HPCProject.cu -o test

In [ ]:
%%time
!./test

Done from CPU
CPU times: user 12.8 ms, sys: 459 µs, total: 13.2 ms
Wall time: 1.32 s
